In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

from sklearn import model_selection
from sklearn import impute, preprocessing
from sklearn import pipeline, compose
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import linear_model, ensemble
import catboost
from sklearn import metrics

## Цель работы

В данной работе построим baseline-модели разного типа и сделаем предположения о том, какие подходы являются наиболее перспективными, чтобы в дальнейшим оценить какой прирост в качества дают различные преобразования, усложнения, оптимизации. 

На разработку baseline-модели не должно уходить много времени, поэтому будем использовать простые решения. В предобработке данных ограничимся удалением константных признаков, заполнением пропусков среднимим значениями у вещественных и самыми популярными значениями у категориальных признаков. Далее сделаем стандартизацию и кодируем категориальные признаки.

В качестве baseline-моделей будем использовать следующие:

 - Логистическую регрессию
 - Случайный лес
 - Градиентный бустинг
 
Оценку качества модели будем производить на основе кросс-валидации. Будем использовать stratified k-fold с разбиением на 4 фолда. В качестве основной метрики оценки качества модели будем использовать ROC-AUC.

## Импортируем данные

In [3]:
data = pd.read_csv('orange_small_churn_data.txt')
labels = pd.read_csv('orange_small_churn_labels.txt', 
                     header = None, names = ['churn'])

Посмотрим на несколько строк данных:

In [4]:
print(data.shape)
data.head(5)

(40000, 230)


,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
0,NaN,NaN,NaN,NaN,NaN,3052.0,NaN,NaN,NaN,NaN,...,Al6ZaUT,vr93T2a,LM8l689qOp,NaN,NaN,fKCe,02N6s8f,xwM2aC7IdeMC0,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,1813.0,7.0,NaN,NaN,NaN,...,oslk,6hQ9lNX,LM8l689qOp,NaN,ELof,xb3V,RAYp,55YFVY9,mj86,NaN
2,NaN,NaN,NaN,NaN,NaN,1953.0,7.0,NaN,NaN,NaN,...,zCkv,catzS2D,LM8l689qOp,NaN,NaN,FSa2,ZI9m,ib5G6X1eUxUn6,mj86,NaN
3,NaN,NaN,NaN,NaN,NaN,1533.0,7.0,NaN,NaN,NaN,...,oslk,e4lqvY0,LM8l689qOp,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,686.0,7.0,NaN,NaN,NaN,...,oslk,MAz3HNj,LM8l689qOp,NaN,NaN,WqMG,RAYp,F2FyR07IdsN7I,NaN,NaN


Разделим признаки на вещественные и категориальные. Признаки, которые имеют одно уникальное значение не будем использовать, т к константные признаки не имеют смысла при построении модели. 

Также не будем использовать категориальные признаки, вариация значений которых больше 300.

In [5]:
def feature_border(data, low, hight, num = True):
    all_features = [''.join(('Var', str(i))) for i in range(low, hight)]
    result_features = []
    for var in all_features:
        size_feature = data[var].unique().shape[0]
        if num:
            if size_feature >= 2:
                result_features.append(var)
        else:
            if size_feature >= 2 and size_feature < 300:
                result_features.append(var)
    return result_features

In [6]:
num_features = feature_border(data, 1, 191)
cat_features = feature_border(data, 191, 231, num = False)

In [7]:
print('num_feaures size: {}'.format(len(num_features)))
print('cat_feaures size: {}'.format(len(cat_features)))

num_feaures size: 174
cat_feaures size: 28


#### Оставим отложенную выборку для тестирования качества модели:

In [8]:
(data_train, data_test, 
 y_train, y_test) = model_selection.train_test_split(data, labels, 
                                                    test_size = 0.20, 
                                                    random_state = 1)


print('data_train size: {}'.format(data_train.shape))
print('labels_train size: {}'.format(y_train.shape))
print('')
print('data_test size: {}'.format(data_test.shape))
print('labels_test size: {}'.format(y_test.shape))

data_train size: (32000, 230)
labels_train size: (32000, 1)

data_test size: (8000, 230)
labels_test size: (8000, 1)


#### Заполним пропуски в данных:
Для вещественных признаков пропуски заполним средним значением по столбцу, а для категориальных константным значением.

In [9]:
num_preprocessing = pipeline.Pipeline(steps = [
    ('num', impute.SimpleImputer())
])


cat_preprocessing = pipeline.Pipeline(steps = [
    ('cat', impute.SimpleImputer(strategy = 'most_frequent'))
])

#### Стандартизуем вещественные признаки и кодируем категориальные: 
Вещественные признаки отличаются друг от друга по модулю значений, поэтому выполним их стандартизацию. Для использования категориальных признаков при построении модели их необходимо преобразовать в вещественные. Будем использовать one-hot encoding.

In [10]:
num_preprocessing.steps.append(
    ('num_scaler', preprocessing.StandardScaler())
)


cat_preprocessing.steps.append(
    ('cat_encoder', preprocessing.OneHotEncoder(handle_unknown = 'ignore'))
)


# трансформер для заполнения пропусков и преобразования признаков  
data_preprocessing = compose.ColumnTransformer(transformers = [
    ('num_features', num_preprocessing, num_features),
    ('cat_features', cat_preprocessing, cat_features)
])

## Построение baseline-моделей

#### Логистическая регрессия:

Классы в данной задаче несбалансированны, поэтому добавим балансировку классов в модель. Остальные параметры оставим по умолчанию.

In [11]:
logistic_regression = pipeline.Pipeline(steps = [
    ('preprocessing', data_preprocessing),
    ('logistic', linear_model.LogisticRegression(class_weight = 'balanced', 
                                                 n_jobs = -1))
])

Качество оценим с помощью кросс-валидации. В качетве вспомогательной метрики выбирем F1 меру.   

In [12]:
def cross_validation(model, data, y, cv = 4, scoring = 'roc_auc'):
    result = model_selection.cross_val_score(model, data,y, 
                               cv = cv, scoring = scoring,  
                               n_jobs = -1)
    return result

In [13]:
cv_skf = model_selection.StratifiedKFold(n_splits = 4, random_state = 1)

roc_auc_score_logistic = cross_validation(logistic_regression, data_train, y_train, 
                               cv = cv_skf, scoring = 'roc_auc')

f1_score_logistic = cross_validation(logistic_regression, data_train, y_train, 
                               cv = cv_skf, scoring = 'f1')

In [14]:
print('roc_auc_score: {}  mean: {}'.format(roc_auc_score_logistic, 
                                          round(roc_auc_score_logistic.mean(), 4)))
print(' ')
print('f1_score: {}  mean: {}'.format(f1_score_logistic, 
                                     round(f1_score_logistic.mean(), 4)))

roc_auc_score: [0.6368168  0.65440807 0.65184991 0.63216506]  mean: 0.6438
 
f1_score: [0.18686869 0.1894393  0.19091904 0.18297052]  mean: 0.1875


#### Случайный лес:

In [15]:
random_forest = pipeline.Pipeline(steps = [
    ('preprocessing', data_preprocessing), 
    ('forest', ensemble.RandomForestClassifier(max_depth = 5,
                                               class_weight = 'balanced',
                                               n_jobs = -1))
])

In [16]:
roc_auc_score_forest = cross_validation(random_forest, data_train, y_train, 
                               cv = cv_skf, scoring = 'roc_auc')

f1_score_forest = cross_validation(random_forest, data_train, y_train, 
                               cv = cv_skf, scoring = 'f1')

In [17]:
print('roc_auc_score: {}  mean: {}'.format(roc_auc_score_forest, 
                                          round(roc_auc_score_forest.mean(), 4)))
print(' ')
print('f1_score: {}  mean: {}'.format(f1_score_forest, 
                                     round(f1_score_forest.mean(), 4)))

roc_auc_score: [0.65845972 0.66509684 0.67088448 0.68177553]  mean: 0.6691
 
f1_score: [0.19291887 0.20099119 0.20132743 0.19994724]  mean: 0.1988


#### Градиентный бустинг:

In [19]:
# numpy array in DataFrame with features name
class DataForCatboost(BaseEstimator, TransformerMixin):
    
    def __init__(self, num_features, cat_features):
        self.num_features = num_features
        self.cat_features = cat_features
        self.data = None
        self.size_data = 0
        
    def fit(self, X, y = None):
        self.size_data = X.shape[0]
        self.data = pd.DataFrame(columns = self.num_features + self.cat_features, 
                                data = X)
        return self
    
    def transform(self, X):
        if self.size_data == X.shape[0] or self.size_data == 0:
            return self.data
        else: 
            current_data = pd.DataFrame(columns = self.num_features + self.cat_features, 
                                data = X)
            return current_data
    
    def fit_transform(self, X, y = None):
        self.fit(X)
        return self.transform(X)

In [20]:
cat_preprocessing_for_boosting = pipeline.Pipeline(steps = [
    ('cat_impute', impute.SimpleImputer(strategy = 'most_frequent'))
])


# трансформер для заполнения пропусков и преобразования вещественных признаков
features_for_catboost = compose.ColumnTransformer(transformers = [
    ('num_features', num_preprocessing, num_features),
    ('cat_features', cat_preprocessing_for_boosting, cat_features)
])


# итоговый pipeline для предобработки данных
all_features = pipeline.Pipeline(steps = [
    ('feature', features_for_catboost), 
    ('data', DataForCatboost(num_features, cat_features))
])

In [21]:
model = catboost.CatBoostClassifier(n_estimators = 100,
                                    max_depth = 5,
                                    class_weights = [0.07, 0.93],
                                    cat_features = cat_features)

In [22]:
def cross_validation_for_catboost(model, feature_transformer, data, y, cv, metrics):
    result = []
    for train_indices, test_indices in cv.split(data, y):
        current_model = model
        transformer = feature_transformer
        # train data and target
        data_train = transformer.fit_transform(data.iloc[train_indices])
        y_train = y.iloc[train_indices]
        # tets data and target
        data_test = transformer.transform(data.iloc[test_indices])
        y_test = y.iloc[test_indices]
        
        # fit on train data and predict on dat test
        current_model.fit(data_train, y_train, verbose = False)
        result.append(metrics(y_test, current_model.predict(data_test)))
    return result

In [23]:
roc_auc_score_catboost = cross_validation_for_catboost(model, all_features, data_train, y_train, 
                                                       cv = cv_skf, 
                                                       metrics = metrics.roc_auc_score)


f1_score_catboost = cross_validation_for_catboost(model, all_features, data_train, y_train, 
                                                  cv = cv_skf, 
                                                  metrics = metrics.f1_score)

In [24]:
print('roc_auc_score: {}  mean: {}'.format(roc_auc_score_catboost, 
                                          round(np.array(roc_auc_score_catboost).mean(), 4)))
print(' ')
print('f1_score: {}  mean: {}'.format(f1_score_catboost, 
                                     round(np.array(f1_score_catboost).mean(), 4)))

roc_auc_score: [0.6291189416930446, 0.6460835403094992, 0.6308057691311255, 0.6374517461004876]  mean: 0.6359
 
f1_score: [0.21364985163204747, 0.22847457627118647, 0.21440208536982733, 0.22117962466487937]  mean: 0.2194


---

## Выводы

**Целью** данной работы было построение baseline-моделей и оценка их качества. 

Были построены слудущие модели:
 - **Логистическая регрессия**:  ROC-AUC = 0.64
 - **Случайный лес**:  ROC-AUC = 0.67
 - **Градиентный бустинг** (catboost): ROC-AUC = 0.64

При построении основного решения упор стоит сделать на модели, основанные на деревьях (случайный лес и градиентный бустинг), т к они показали лучшее качество.

На всех моделях вспомогательня метрика качества **f1-мера** очень маленькая. Определяется **f1-мера** как среднее гармоническое точности и полноты, поэтому стоит отдельно посмотреть на эти них.

Воспользуемся случайным лесом и оценим **точность** и **полноту** на кросс-валидации:

In [31]:
precision_forest = cross_validation(random_forest, data_train, y_train, 
                               cv = cv_skf, scoring = 'precision')

recall_forest = cross_validation(random_forest, data_train, y_train, 
                               cv = cv_skf, scoring = 'recall')

In [38]:
print('precision mean: {}'.format(round(precision_forest.mean(), 4)))
print(' ')
print('recall mean: {}'.format(round(recall_forest.mean(), 4)))

precision mean: 0.119
 
recall mean: 0.6033


Имеем очень низкое значение **точности**. Значит наш алгоритм совершает много ошибок **false positive**, т е происходит ложные срабатывания (относим объект к классу "отток", при его истином значении класса "не отток". 

В нашей задаче хуже совершать ошибки **false negative**, т е относить объект к классу "не отток" при его истинном значении класса "отток". Это приведет к потере клиентов, т к мы будем пропускать клиентов, которые хотят осуществить переход конкуренту и не сможем применить к ним методы для удержания.